<a href="https://colab.research.google.com/github/Mano-2410/pulmonary-Nodule-detection-and-Classification-using-deep-learning-algorithm/blob/Master/PulmonaryLungNoduleDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import measure
from tqdm import tqdm
import random
import copy

In [ ]:
dataRoot = "/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/models/v1/UNet_model.h5", custom_objects={'dice_coef':dice_coef, 'dice_coef_loss':dice_coef_loss})
model2 = tf.keras.models.load_model("/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/models/v1/FPR_classifier_model.h5")

In [ ]:
import nbformat as nbf
def merge_notebooks(filenames):
    merged = nbf.v4.new_notebook()
    for fname in filenames:
        with open(fname) as f:
            nb = nbf.read(f, as_version=4)
            merged.cells.extend(nb.cells)
    return merged

notebooks_to_merge = ["/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/01_Lungs_Segmentation.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/02_Nodule_Mask_Extraction.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/03_v2_Lungs ROI & Nodule Mask extraction from LUNA16 dataset.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/04_Dataset_Preparation.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/05_UNet_Training.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/06_Ground_Truth_vs_Prediction_visualization.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/07_FPR_Candidate_ROI_extraction.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/08_FPR_Dataset_Creation.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/09_FPR_CNN_Training.ipynb",
"/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/10_LC_FPR_CNN_Evaluation.ipynb","/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/notebooks/v1/FPR_data_lookup.ipynb"]

merged_notebook = merge_notebooks(notebooks_to_merge);
with open("merged_notebook.ipynb", "w") as f:
    nbf.write(merged_notebook, f)



/usr/local/lib/python3.10/dist-packages/nbformat/__init__.py:132: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
!jupyter notebook merged_notebook.ipynb

|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/usr/local/etc/jupyter/jupyter_notebook_config.d/ipyparallel.json
    	/usr/local/etc/jupyter/jupyter_notebook_config.d/panel-client-jupyter.json
    	/usr/local/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/usr/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/root/.local/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/root/.jupyter/jupyter_notebook_config.json

  _   _          _      _
 | | | |_ __  __| |__ _| |_ ___
 | |_| | '_ \/ _` / _` |  _/ -_)
  \___/| .__/\__,_\__,_|\__\___|
       |_|
                       
Read the migration plan to Notebook 7 to learn about the new features and the actions to take if you 

In [ ]:
mfiles = os.listdir(dataRoot+"/Malignant cases/")
random.shuffle(mfiles)
mfiles = mfiles[:30]
bfiles = os.listdir(dataRoot+"/Bengin cases/")
random.shuffle(bfiles)
bfiles = bfiles[:30]
nfiles = os.listdir(dataRoot+"/Normal cases/")
random.shuffle(nfiles)
nfiles = nfiles[:30]

In [ ]:
def preprocess(img):
    if img.max()>255 or img.max()<1.5 or img.min()<0:
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (512,512))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_improved = clahe.apply(img.astype(np.uint8))
    # img_improved = img.copy()

    contours, hirearchy = cv2.findContours(external_contours,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    external_contours = np.zeros(external_contours.shape)
    centeral_area = img[100:400, 100:400]
    kmeans = KMeans(n_clusters=2).fit(np.reshape(centeral_area, [np.prod(centeral_area.shape), 1]))
    centroids = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centroids)
    ret, lung_roi = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY_INV)
    lung_roi = cv2.erode(lung_roi, kernel=np.ones([4,4]))
    lung_roi = cv2.dilate(lung_roi, kernel=np.ones([13,13]))
    lung_roi = cv2.erode(lung_roi, kernel=np.ones([8,8]))

    labels = measure.label(lung_roi)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0] < 500 and B[3]-B[1] < 490 and B[0] > 17 and B[2] < 495:
            good_labels.append(prop.label)
    lung_roi_mask = np.zeros_like(labels)
    for N in good_labels:
        lung_roi_mask = lung_roi_mask + np.where(labels == N, 1, 0)

    contours, hirearchy = cv2.findContours(lung_roi_mask,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    external_contours = np.zeros(lung_roi_mask.shape)
    for i in range(len(contours)):
        if hirearchy[0][i][3] == -1:  #External Contours
            area = cv2.contourArea(contours[i])
            if area>518.0:
                cv2.drawContours(external_contours,contours,i,(1,1,1),-1)
    external_contours = cv2.dilate(external_contours, kernel=np.ones([4,4]))

    external_contours = cv2.bitwise_not(external_contours.astype(np.uint8))
    external_contours = cv2.erode(external_contours, kernel=np.ones((7,7)))
    external_contours = cv2.bitwise_not(external_contours)
    external_contours = cv2.dilate(external_contours, kernel=np.ones((12,12)))
    external_contours = cv2.erode(external_contours, kernel=np.ones((12,12)))

    contours, hirearchy = cv2.findContours(external_contours,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    external_contours2 = np.zeros(external_contours.shape)
    for i in range(len(contours)):
        if hirearchy[0][i][3] == -1:  #External Contours
            area = cv2.contourArea(contours[i])
            if area>518.0:
                cv2.drawContours(external_contours2,contours,i,(1,1,1),-1)

    img_improved = img_improved.astype(np.uint8)
    external_contours2 = external_contours2.astype(np.uint8)
    extracted_lungs = cv2.bitwise_and(img_improved, img_improved, mask=external_contours2)

    return ((extracted_lungs-127.0)/127.0).astype(np.float32)

In [ ]:
def display(imgs,mask):
    j = 1
    for im,mk in zip(imgs,mask):
      plt.figure(figsize=(20,300))
      plt.subplot(50,3,j)
      plt.imshow(np.squeeze(im), cmap="gray")
      plt.subplot(50,3,j+1)
      plt.imshow(np.squeeze(mk), cmap="gray")
      plt.subplot(50,3,j+2)
      plt.imshow(cv2.addWeighted(np.squeeze(im), .5, np.squeeze(mk), .5, 0), cmap="gray")
      j += 3

In [ ]:
def display2(imgs, titles=None, cmap="bone"):
    n = len(list(imgs))
    r = n//3 if n%3==0 else (n//3)+1
    plt.figure(figsize=(25,int(8*r)))
    for i,img in enumerate(imgs):
        plt.subplot(r,3,i+1)
        if titles is not None:
            plt.title(titles[i])
        plt.imshow(img, cmap=cmap)

In [ ]:
def display_test(img, cmap='bone'):
    plt.figure(figsize=(10,10))
    plt.imshow(img, cmap=cmap)

In [ ]:
originals = []
mimgs = []
bimgs = []
nimgs = []

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
for m in tqdm(mfiles):
    img = cv2.imread(dataRoot+"/Malignant cases/"+m, 0)
    originals.append(clahe.apply(cv2.resize(img, (512,512)).astype(np.uint8)))
    img = preprocess(img)
    mimgs.append(img)


for m in tqdm(bfiles):
    img = cv2.imread(dataRoot+"Bengin cases/"+m, 0)
    img = preprocess(img)
    bimgs.append(img)

for m in tqdm(nfiles):
    img = cv2.imread(dataRoot+"Normal cases/"+m, 0)
    img = preprocess(img)
    nimgs.append(img)

In [ ]:
mi = np.array(mimgs)
pred = model.predict(np.reshape(mi,(len(mi),512,512,1)))
pred[pred>=0.5] = 255
pred[pred<0.5] = 0
pred = pred.astype(np.uint8)
pred = list(pred)
mpred = [np.squeeze(i) for i in pred]

# bi = np.array(bimgs)
# pred = model.predict(np.reshape(bi,(len(bi),512,512,1)))
# pred = list(pred)
# bpred = [np.squeeze(i) for i in pred]

# ni = np.array(nimgs)
# pred = model.predict(np.reshape(ni,(len(ni),512,512,1)))
# pred = list(pred)
# npred = [np.squeeze(i) for i in pred]

In [ ]:
display(originals[10:15],mpred[10:15])

In [ ]:
bboxes = []
for mask in mpred:
    mask = cv2.dilate(mask, kernel=np.ones((5,5)))
    labels = measure.label(mask)
    regions = measure.regionprops(labels)
    bb = []
    for prop in regions:
        B = prop.bbox
        bb.append((( max(0, B[1]-10), max(0, B[0]-10) ),( min(B[3]+10, 512), min(B[2]+10, 512) )))    # ((x1,y1),(x2,y2))
    bboxes.append(bb)

In [ ]:
bboxes[1]

In [ ]:
bs = []
o = copy.deepcopy(mimgs)
for i,(img,boxes) in enumerate(zip(o,bboxes)):
    for rect in boxes:
        cv2.rectangle(img, rect[0], rect[1], (1), 2)

In [ ]:
display2(o[10:16])

In [ ]:
o2 = copy.deepcopy(originals)
final_boxes = []
for i,(img,bbox) in enumerate(zip(o2, bboxes)):
    img_boxes = []
    for box in bbox:
        x1 = box[0][0]
        y1 = box[0][1]
        x2 = box[1][0]
        y2 = box[1][1]
        if abs(x1-x2) <=50 or abs(y1-y2)<=50:
            x = (x1+x2)//2
            y = (y1+y2)//2
            x1 = max(x-25, 0)
            x2 = min(x+25, 512)
            y1 = max(y-25, 0)
            y2 = min(y+25, 512)
            imgbox = img[y1:y2,x1:x2]
            img_boxes.append(imgbox)
        else:
            imgbox = img[y1:y2,x1:x2]
            img_boxes.append(imgbox)
    final_boxes.append(img_boxes)

In [ ]:
plt.figure(figsize=(15,15))
for i,img in enumerate(final_boxes[13]):
    plt.subplot(1,6,i+1)
    plt.imshow(img, cmap="gray")

In [ ]:
predictions = []
for i in final_boxes:
    each_p = []
    for img in i:
        if img.shape != (50,50):
            img = np.resize(img, (50,50))
        img = img/255.
        img = np.reshape(img, (1,50,50,1))
        pred = model2.predict(img)
        pred = pred>=0.5
        each_p.append(1)
    predictions.append(each_p)

In [ ]:
final_img_bbox = []
cancer = []
for i,(img,bbox,preds) in enumerate(zip(mimgs, bboxes, predictions)):
    token = False
    for box,pred in zip(bbox,preds):
        if pred:
            x1 = box[0][0]
            y1 = box[0][1]
            x2 = box[1][0]
            y2 = box[1][1]
            img = cv2.rectangle(img, (x1,y1), (x2,y2), (2), 2)
            token = True
    final_img_bbox.append(img)
    cancer.append(token)

In [ ]:
display2(final_img_bbox[10:16], cancer[10:16])

# Training YOLOv5 on costume dataset Lung cancer Nodules detection

In this paper , we assemble a dataset and train a custom YOLOv5 model to recognize the Nodules in our dataset. To do so we will take the following steps:

* Gather a dataset of images and label our dataset from LIDC databased
* Export our dataset to YOLOv5
* Train YOLOv5 to recognize the lung cancer Nodules annatationed in our dataset
* Evaluate our YOLOv5 model's performance
* Run test inference to view our model perfermence on wandb W and B



![](https://miro.medium.com/max/1398/1*ByicK9w-I8wdNGfdx8DJZA.png)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Step 1: Install Requirements

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

In [ ]:
import torch
import os
from IPython.display import Image, clear_output  # to display images
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

### Create Config Data File

* the file config Parametes :
    - config Data :
        1. the data Path
        2. Train Path
        3. validation Path
        4. Number Claess
        5. name of Class
        
**Noatation: we will let the hyper-Parameters as defined in Yolov5s**

In [ ]:
import yaml
# Define the YAML content as a Python dictionary
data = {
    'path': '/content/drive/MyDrive/ct_images',
    'train': 'images/train',
    'val': 'images/val',
    'test': None,  # Optional key, set to None for now
    'nc': 1,
    'names': ['Nodules']
}

# Specify the file path for the new YAML file
file_path = '/content/dataconfig.yml'

# Write the YAML content to the file
with open(file_path, 'w') as file:
    yaml.dump(data, file)

In [ ]:
!cat /content/output/yolov5/dataconfig.yml

In [ ]:
%%capture
!pip install comet_ml  # 1. install

In [ ]:
!export COMET_API_KEY=YB7WECZkESGlU5uxxO43BCo5Q  # 2. paste API key
!python /content/yolov5/train.py --img 50 --batch 16 --single-cls --epochs 3 --project cancer_Nodules_diagnosed --bbox_interval 1 --save-period 4 --data /content/dataconfig.yml --weights yolov5s.pt  # 3. train

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/yolov5/cancer_Nodules_diagnosed/exp

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
!python /content/yolov5/val.py --weights /content/yolov5/cancer_Nodules_diagnosed/exp/weights/best.pt --single-cls --img 416 --conf 0.1 --data /content/dataconfig.yml

In [ ]:
!python /content/yolov5/detect.py --weights /content/cancer_Nodules_diagnosed/exp/weights/best.pt --img 416 --conf 0.1 --source /content/drive/MyDrive/ct_images/images/val

In [ ]:
import glob
import matplotlib.pyplot as plt

# Get the list of image file paths
image_paths = glob.glob('/content/yolov5/runs/detect/exp4/*.jpg')

# Plot only the first six images
num_images_to_plot = 6

# Initialize the figure and axes
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for i, (image_path, ax) in enumerate(zip(image_paths[:num_images_to_plot], axes.flatten())):
    # Read and plot the image using Matplotlib
    img = plt.imread(image_path)
    ax.imshow(img)
    ax.axis('off')

    # Display the figure size information
    ax.set_title('Lung Nodule Detection')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()


# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize your custom objects.

To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

In [ ]:
%cd ..

In [ ]:
import shutil
source_file_path = '/content/yolov5/yolov5/cancer_Nodules_diagnosed/exp/weights/best.pt'

destination_directory = 'weights/'
os.makedirs(destination_directory, exist_ok=True)

destination_file_path = destination_directory

# Copy the file to the destination
shutil.copy2(source_file_path, destination_file_path)


In [ ]:
import pickle
import torch

checkpoint = torch.load('/content/yolov5/weights/best.pt', map_location=torch.device('cpu'), pickle_module=pickle)


In [ ]:
import os

def delete_files_in_dir(directory):
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            # Check if the path is a file and delete it
            if os.path.isfile(file_path):
                os.unlink(file_path)
                print(f"Deleted {file_path}")
        except Exception as e:
            print(f"Failed to delete {file_path}: {e}")

# Specify the directory from which you want to delete files
directory_to_delete = "/content/yolov5/weights"

# Call the function to delete files in the specified directory
print(directory_to_delete)
delete_files_in_dir(directory_to_delete)

Importing Modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install SimpleITK
import os
import cv2
import copy
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

import SimpleITK as stk
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import backend as K

from sklearn.cluster import KMeans
from skimage import measure

In [ ]:
!pip install nbformat
import nbformat as nbf
def merge_notebooks(filenames):
    merged = nbf.v4.new_notebook()
    for fname in filenames:
        with ope
n(fname) as f:
            nb = nbf.read(f, as_version=4)
            merged.cells.extend(nb.cells)
    return merged

notebooks_to_merge =["/content/drive/MyDrive/Colab Notebooks/Test_Final_Prediction.ipynb",
                     "/content/drive/MyDrive/Colab Notebooks/yolov5-on-costume-dataset-lung-cancer-nodules-dete.ipynb",
                     "/content/drive/MyDrive/Colab Notebooks/custom_ct_scan.ipynb"]

merged_notebook = merge_notebooks(notebooks_to_merge);
with open("/content/drive/MyDrive/Colab Notebooks/PulmonaryLungNoduleDetection.ipynb", "w") as f:
    nbf.write(merged_notebook, f)


Path for Data

In [ ]:
PATH = "/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/DATA"

In [ ]:
FILE = "/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/imgs/Testimg1.jpg"

Function to load mhd files

In [ ]:
def load_mhd(file):
    mhdimage = stk.ReadImage(file)
    ct_scan = stk.GetArrayFromImage(mhdimage)
    origin = np.array(list(mhdimage.GetOrigin()))
    space = np.array(list(mhdimage.GetSpacing()))
    return ct_scan, origin, space

Custom Evaluation metrics for U-Net model

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

Loading U-Net model

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Untitled folder/Lung_Cancer_Detection-main/models/v2/UNet_model.h5", custom_objects={'dice_coef':dice_coef, 'dice_coef_loss':dice_coef_loss})

Read Custom CT-Scan (mhd file)

In [ ]:
#ct, origin, space = load_mhd(FILE)
ct = cv2.imread(FILE)
print(ct.shape)

In [ ]:
num_z, width,height = ct.shape
ct_norm = ct.astype(np.uint8)
#ct_norm = cv2.normalize(ct, None, 0, 1, cv2.NORM_MINMAX)
ct_norm = cv2.cvtColor(ct, cv2.COLOR_RGB2GRAY)   # Normalizing the CT scan


In [ ]:
#plt.figure(figsize=(width,height))
##plt.imshow(ct)
plt.imshow(ct_norm)
#plt.imshow(ct_norm[57], cmap="gray", interpolation="nearest")

In [ ]:
ct_norm = cv2.resize(ct_norm, (512,512))
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
ct_norm_improved = clahe.apply(ct_norm.astype(np.uint8))

In [ ]:
plt.imshow(ct_norm_improved, cmap = "gray")

In [ ]:
centeral_area = ct_norm_improved[100:400, 100:400]
kmeans = KMeans(n_clusters=2).fit(np.reshape(centeral_area, [np.prod(centeral_area.shape), 1]))
centroids = sorted(kmeans.cluster_centers_.flatten())
threshold = np.mean(centroids)
print(threshold)

In [ ]:
    ret, lung_roi = cv2.threshold(ct_norm, threshold, 255, cv2.THRESH_BINARY_INV)
    lung_roi = cv2.erode(lung_roi, kernel=np.ones([4,4]))
    lung_roi = cv2.dilate(lung_roi, kernel=np.ones([13,13]))
    lung_roi = cv2.erode(lung_roi, kernel=np.ones([8,8]))

    labels = measure.label(lung_roi)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0] < 500 and B[3]-B[1] < 490 and B[0] > 17 and B[2] < 495:
            good_labels.append(prop.label)
    lung_roi_mask = np.zeros_like(labels)
    for N in good_labels:
        lung_roi_mask = lung_roi_mask + np.where(labels == N, 1, 0)

    contours, hirearchy = cv2.findContours(lung_roi_mask,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    external_contours = np.zeros(lung_roi_mask.shape)
    for i in range(len(contours)):
        if hirearchy[0][i][3] == -1:  #External Contours
            area = cv2.contourArea(contours[i])
            if area>518.0:
                cv2.drawContours(external_contours,contours,i,(1,1,1),-1)
    external_contours = cv2.dilate(external_contours, kernel=np.ones([4,4]))

    external_contours = cv2.bitwise_not(external_contours.astype(np.uint8))
    external_contours = cv2.erode(external_contours, kernel=np.ones((7,7)))
    external_contours = cv2.bitwise_not(external_contours)
    external_contours = cv2.dilate(external_contours, kernel=np.ones((12,12)))
    external_contours = cv2.erode(external_contours, kernel=np.ones((12,12)))


    contours, hirearchy = cv2.findContours(external_contours,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    external_contours2 = np.zeros(external_contours.shape)
    lungs_mask =
    cv2.resize(external_contours2, ct_norm_improved.shape)
    lungs_mask = external_contours2
    for i in range(len(contours)):
        if hirearchy[0][i][3] == -1:  #External Contours
            area = cv2.contourArea(contours[i])
            if area>518.0:
                cv2.drawContours(external_contours2,contours,i,(1,1,1),-1)



In [ ]:
#!pip install flask gunicorn pyngrok

from flask import Flask, request
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)

# Define prediction route
@app.route('/', methods=['POST'])
def predict():
    # Your ML model prediction logic here
    data = request.json
    prediction = model.predict(data)
    return {'prediction': prediction}

# Run Flask app with Gunicorn
def run_flask_with_gunicorn():
    !gunicorn -w 4 -b 127.0.0.1:8000 app:app &>/dev/null &

# Expose Gunicorn server with Ngrok
def expose_with_ngrok():
    public_url = ngrok.connect(port='8000')
    print('Ngrok URL:', public_url)

if __name__ == '__main__':
    run_flask_with_gunicorn()
    expose_with_ngrok()


In [ ]:
plt.imshow(lungs_mask, cmap="gray")

In [ ]:
#extracted_lungs = (cv2.bitwise_and(ct_norm_improved, ct_norm_improved, mask=mask))

In [ ]:
plt.imshow(extracted_lungs, cmap="bone")

In [ ]:
X = np.array(extracted_lungs)
X.shape

In [ ]:
X = (X-127.0)/127.0
X = X.astype(np.float32)
X.dtype

In [ ]:
print("Min:",X.min(),"\nMax:",X.max())

In [ ]:
X = np.reshape(X, (len(X), 512, 512, 1))
X.shape

In [ ]:
predictions = model.predict(X)
predictions.shape

In [ ]:
print("Dtype:",predictions.dtype,"\nMin:",predictions.min(),"\nMax:",predictions.max())

In [ ]:
predictions[predictions>=0.5] = 255
predictions[predictions<0.5] = 0
predictions = predictions.astype(np.uint8)

In [ ]:
pred = list(predictions)
pred = [np.squeeze(i) for i in pred]

In [ ]:
plt.imshow(pred[61], cmap="gray")

In [ ]:
def display(imgs,mask):
    j = 1
    for im,mk in zip(imgs,mask):
        plt.figure(figsize=(20,300))
        plt.subplot(50,3,j)
        plt.imshow(np.squeeze(im), cmap="gray")
        plt.subplot(50,3,j+1)
        plt.imshow(np.squeeze(mk), cmap="gray")
        plt.subplot(50,3,j+2)
        plt.imshow(cv2.addWeighted(np.squeeze(im), .5, np.squeeze(mk), .5, 0), cmap="gray")
        j += 3

In [ ]:
def display2(imgs, titles=None, cmap="bone"):
    n = len(list(imgs))
    r = n//3 if n%3==0 else (n//3)+1
    plt.figure(figsize=(25,int(8*r)))
    for i,img in enumerate(imgs):
        plt.subplot(r,3,i+1)
        if titles is not None:
            plt.title(titles[i])
        plt.imshow(img, cmap=cmap)

In [ ]:
display(extracted_lungs[65:70],pred[65:70])

In [ ]:
bboxes = []
centroids = []
diams = []
for mask in pred:
    mask = cv2.dilate(mask, kernel=np.ones((5,5)))
    labels = measure.label(mask)
    regions = measure.regionprops(labels)
    bb = []
    cc = []
    dd = []
    for prop in regions:
        B = prop.bbox
        C = prop.centroid
        D = prop.equivalent_diameter_area
        bb.append((( max(0, B[1]-8), max(0, B[0]-8) ),( min(B[3]+8, 512), min(B[2]+8, 512) )))    # ((x1,y1),(x2,y2))
        cc.append(C)    # (y,x)
        dd.append(D)
    bboxes.append(bb)
    centroids.append(cc)
    diams.append(dd)

In [ ]:
bboxes[65:71]

In [ ]:
centroids[65:71]

In [ ]:
diams[65:71]

In [ ]:
bs = []
mimgs = copy.deepcopy(extracted_lungs)
for i,(img,boxes) in enumerate(zip(mimgs,bboxes)):
    for rect in boxes:
        img = cv2.rectangle(img, rect[0], rect[1], (255), 2)

In [ ]:
display2(mimgs[40:55])

In [ ]:
idx = 66
v = copy.deepcopy(extracted_lungs)
plt.figure(figsize=(10,10))
i = cv2.rectangle(v[idx], bboxes[idx][0][0], bboxes[idx][0][1], (255), 2)
i = cv2.circle(i, (int(centroids[idx][0][1]), int(centroids[idx][0][0])), 2, (255), -1)
plt.imshow(i, cmap='gray')

In [ ]:
diams[66][0]*space[0]   # diameter in mm

In [ ]:
fpr_model = tf.keras.models.load_model("models/FPR_classifier_model.h5")

In [ ]:
originals = copy.deepcopy(ct_norm_improved)
final_boxes = []
for i,(img,bbox) in enumerate(zip(originals, bboxes)):
    img_boxes = []
    for box in bbox:
        x1 = box[0][0]
        y1 = box[0][1]
        x2 = box[1][0]
        y2 = box[1][1]
        if abs(x1-x2) <=50 or abs(y1-y2)<=50:
            x = (x1+x2)//2
            y = (y1+y2)//2
            x1 = max(x-25, 0)
            x2 = min(x+25, 512)
            y1 = max(y-25, 0)
            y2 = min(y+25, 512)
            imgbox = img[y1:y2,x1:x2]
            img_boxes.append(imgbox)
        else:
            imgbox = img[y1:y2,x1:x2]
            img_boxes.append(imgbox)
    final_boxes.append(img_boxes)

In [ ]:
plt.figure(figsize=(15,15))
for i,img in enumerate(final_boxes[66]):
    plt.subplot(1,6,i+1)
    plt.imshow(img, cmap="gray")

In [ ]:
fpr_preds = []
for i in final_boxes:
    each_p = []
    for img in i:
        if img.shape != (50,50):
            img = np.resize(img, (50,50))
        img = img/255.
        img = np.reshape(img, (1,50,50,1))
        pred = fpr_model.predict(img)
        pred = int(pred>=0.5)
        each_p.append(pred)
    fpr_preds.append(each_p)

In [ ]:
bboxes[66]

In [ ]:
for i in range(len(diams)):
    if len(diams[i]):
        for j in range(len(diams[i])):
            diams[i][j] = diams[i][j]*space[0]       # diameters in mm

In [ ]:
final_img_bbox = []
cancer = []
df = pd.DataFrame(columns = ['Layer', 'Position (x,y)', 'Diameter (mm)', 'BBox [(x1,y1),(x2,y2)]'])
e_lungs = copy.deepcopy(ct_norm_improved)
for i,(img,bbox,preds,cents,dms) in enumerate(zip(e_lungs, bboxes, fpr_preds, centroids, diams)):
    token = False
    for box,pred,cent,dm in zip(bbox,preds,cents,dms):
        if pred:
            x1 = box[0][0]
            y1 = box[0][1]
            x2 = box[1][0]
            y2 = box[1][1]
            img = cv2.rectangle(img, (x1,y1), (x2,y2), (255), 2)
            dct = pd.DataFrame({'Layer':i, 'Position (x,y)':[f"{cent[::-1]}"], 'Diameter (mm)':dm, 'BBox [(x1,y1),(x2,y2)]':[f"{list(box)}"]})
            df = pd.concat([df,dct], ignore_index = True)
            token = True
    final_img_bbox.append(img)
    cancer.append(token)

In [ ]:
display2(final_img_bbox[60:72])

In [ ]:
cancer[60:72]

In [ ]:
df = df.reset_index(drop=True)
df.head()

In [ ]:
folder = FILE.replace(".mhd", "")
os.mkdir(f"results/{folder}")
df.to_csv(f"results/{folder}/detections.csv", index=False)

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
vid = cv2.VideoWriter(f"results/{folder}/detections.mp4", fourcc, 5.0, (512,512), False)

In [ ]:
for i in range(len(final_img_bbox)):
    img = final_img_bbox[i].copy()
    img = cv2.putText(img, f"Layer: {i}", (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255), 2)
    vid.write(img)
vid.release()